In [9]:
import os
import sys

import litellm
import mlflow
import websockets
from dotenv import load_dotenv
from mlflow.genai import scorer
from mlflow.genai.scorers import Correctness, Guidelines

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)


In [4]:
load_dotenv()

if not os.getenv('OPENAI_API_KEY'):
    raise ValueError("Please set OPENAI_API_KEY environment variable")

print('OPENAI_API_KEY=' + os.getenv('OPENAI_API_KEY')[:20] + '...')


OPENAI_API_KEY=sk-proj-ZhcCqw0r70dP...


In [ ]:
# This will create a new experiment called "GenAI Evaluation Quickstart" and set it as active
mlflow.set_experiment("GenAI Evaluation Quickstart")


In [5]:
def qa_predict_fn(question: str) -> str:
    """Simple Q&A prediction function"""
    response = litellm.completion(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "system",
                "content": "You are a helpful assistant. Answer questions concisely.",
            },
            {"role": "user", "content": question},
        ],
    )
    return response.choices[0].message.content


In [7]:
# Define a simple Q&A dataset with questions and expected answers
eval_dataset = [
    {
        "inputs": {"question": "What is the capital of France?"},
        "expectations": {"expected_response": "Paris"},
    },
    {
        "inputs": {"question": "Who was the first person to build an airplane?"},
        "expectations": {"expected_response": "Wright Brothers"},
    },
    {
        "inputs": {"question": "Who wrote Romeo and Juliet?"},
        "expectations": {"expected_response": "William Shakespeare"},
    },
]


In [11]:
@scorer
def is_concise(outputs: str) -> bool:
    """Evaluate if the answer is concise (less than 5 words)"""
    return len(outputs.split()) <= 5


scorers = [
    Correctness(),
    Guidelines(name="is_english", guidelines="The answer must be in English"),
    is_concise,
]


In [12]:
results = mlflow.genai.evaluate(
    data=eval_dataset,
    predict_fn=qa_predict_fn,
    scorers=scorers,
)

2025/09/21 18:04:24 INFO mlflow.models.evaluation.utils.trace: Auto tracing is temporarily enabled during the model evaluation for computing some metrics and debugging. To disable tracing, call `mlflow.autolog(disable=True)`.
2025/09/21 18:04:24 INFO mlflow.genai.utils.data_validation: Testing model prediction with the first sample in the dataset.
18:04:26 - LiteLLM Proxy:ERROR: guardrail_registry.py:102 - Could not import litellm.proxy.guardrails.guardrail_hooks.aim: No module named 'websockets'
ERROR:LiteLLM Proxy:Could not import litellm.proxy.guardrails.guardrail_hooks.aim: No module named 'websockets'


MlflowException: API request to http://localhost:5001/api/2.0/mlflow/runs/create failed with exception HTTPConnectionPool(host='localhost', port=5001): Max retries exceeded with url: /api/2.0/mlflow/runs/create (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x3289447d0>: Failed to establish a new connection: [Errno 61] Connection refused'))